In [ ]:
import os
import sys
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import torch

sys.path.append(os.path.dirname(os.path.abspath('')))
import utils.YOLO_utils as YOLO
import utils.FPHA_utils as FPHA
from utils.lmdb_utils import *
from utils.eval_utils import *
from utils.image_utils import *
from utils.json_utils import parse
from models import get_model


In [ ]:
epoch = 80
exp = 'reg_base'
REORDER = FPHA.REORDER_IDX

# Train

In [ ]:
data_split = 'train'
save_prefix = data_split + '_fpha_root'
keys = get_keys(os.path.join(YOLO.FPHA_DIR, save_prefix + "_keys_cache.p"))
bbox_gt = read_all_lmdb_dataroot(keys, os.path.join(YOLO.FPHA_DIR, save_prefix + "_bbox.lmdb"), "float32", 4)
pred_file = os.path.join(YOLO.FPHA_DIR, exp, 'predict_{}_{}.txt'.format(epoch, data_split))
bbox_pred = np.loadtxt(pred_file)
bbox_pred = np.asarray([bbox[:5] for bbox in bbox_pred]) # take only the first bbox

dataroot = os.path.join(YOLO.FPHA_DIR, save_prefix + '_xyz_gt.lmdb')
xyz_gt = read_all_lmdb_dataroot(keys, dataroot, 'float32', (21, 3))
uvd_gt = FPHA.xyz2uvd_color(xyz_gt)
pred_file = os.path.join(YOLO.FPHA_DIR, exp, 'predict_{}_{}_uvd.txt'.format(epoch, data_split))
pred_uvd = np.reshape(np.loadtxt(pred_file), (-1, 21, 3))
pred_uvd = scale_points_WH(pred_uvd, (1, 1), (1920, 1080))
pred_uvd[..., 2] *= 1000
pred_xyz = FPHA.uvd2xyz_color(pred_uvd)

In [ ]:
idx = 42655  
img = np.asarray(Image.open(os.path.join(FPHA.DIR, 'Video_files', keys[idx])))
# img = resize_img(img, (13, 13))
print(keys[idx])
fig, ax = plt.subplots()
ax.imshow(img)

FPHA.draw_bbox(ax, bbox_pred[idx], (img.shape[1], img.shape[0]))
FPHA.draw_bbox(ax, bbox_gt[idx], (img.shape[1], img.shape[0]), 'b')

fig, ax = plt.subplots(1,2, figsize=(15,15))
ax[0].imshow(img)
ax[0].set_title('pred')
FPHA.visualize_joints_2d(ax[0], pred_uvd[idx][REORDER], joint_idxs=False)
ax[1].imshow(img)
ax[1].set_title('true')
FPHA.visualize_joints_2d(ax[1], uvd_gt[idx][REORDER], joint_idxs=False)

In [ ]:
iou_thresh = 0.5
correct = 0
avg_iou = 0
for i in range(len(bbox_pred)):
    iou = YOLO.bbox_iou(bbox_pred[i], bbox_gt[i])
    avg_iou += iou
    if iou > iou_thresh:
        correct += 1
        
recall = correct/len(bbox_pred)
avg_iou = avg_iou/len(bbox_pred)
print('Recall:', recall, 'Avg_IOU:', avg_iou)

In [ ]:
print('%s UVD mean_l2_error: ' %data_split, mean_L2_error(uvd_gt, pred_uvd))
print('%s XYZ mean_l2_error: ' %data_split, mean_L2_error(xyz_gt, pred_xyz))
error = []
for i, (pred, uvd) in enumerate(zip(pred_uvd, uvd_gt)):
    error.append(mean_L2_error(uvd, pred))
error = np.asarray(error)
min_error_idx = np.argmin(error)
max_error_idx = np.argmax(error)
print('Best Pose id:', min_error_idx, 'uvd_error:', error[min_error_idx])
print('Worst Pose id:', max_error_idx, 'uvd_error:', error[max_error_idx])

In [ ]:
pck = percentage_frames_within_error_curve(xyz_gt, pred_xyz)
print(pck)
thresholds = np.arange(0, 85, 5)
print('AUC:', calc_auc(pck, thresholds))

# Test

In [ ]:
data_split = 'test'
save_prefix = data_split + '_fpha_root'
keys = get_keys(os.path.join(YOLO.FPHA_DIR, save_prefix + "_keys_cache.p"))
bbox_gt = read_all_lmdb_dataroot(keys, os.path.join(YOLO.FPHA_DIR, save_prefix + "_bbox.lmdb"), "float32", 4)
pred_file = os.path.join(YOLO.FPHA_DIR, exp, 'predict_{}_{}.txt'.format(epoch, data_split))
bbox_pred = np.loadtxt(pred_file)
bbox_pred = np.asarray([bbox[:5] for bbox in bbox_pred]) # take only the first bbox

dataroot = os.path.join(YOLO.FPHA_DIR, save_prefix + '_xyz_gt.lmdb')
xyz_gt = read_all_lmdb_dataroot(keys, dataroot, 'float32', (21, 3))
uvd_gt = FPHA.xyz2uvd_color(xyz_gt)
pred_file = os.path.join(YOLO.FPHA_DIR, exp, 'predict_{}_{}_uvd.txt'.format(epoch, data_split))
pred_uvd = np.reshape(np.loadtxt(pred_file), (-1, 21, 3))
pred_uvd = scale_points_WH(pred_uvd, (1, 1), (1920, 1080))
pred_uvd[..., 2] *= 1000
pred_xyz = FPHA.uvd2xyz_color(pred_uvd)

In [ ]:
idx = 47526  
img = np.asarray(Image.open(os.path.join(FPHA.DIR, 'Video_files', keys[idx])))
# img = resize_img(img, (13, 13))
print(keys[idx])
fig, ax = plt.subplots()
ax.imshow(img)

FPHA.draw_bbox(ax, bbox_pred[idx], (img.shape[1], img.shape[0]))
FPHA.draw_bbox(ax, bbox_gt[idx], (img.shape[1], img.shape[0]), 'b')

fig, ax = plt.subplots(1,2, figsize=(15,15))
ax[0].imshow(img)
ax[0].set_title('pred')
FPHA.visualize_joints_2d(ax[0], pred_uvd[idx][REORDER], joint_idxs=False)
ax[1].imshow(img)
ax[1].set_title('true')
FPHA.visualize_joints_2d(ax[1], uvd_gt[idx][REORDER], joint_idxs=False)

In [ ]:
iou_thresh = 0.5
correct = 0
avg_iou = 0
for i in range(len(bbox_pred)):
    iou = YOLO.bbox_iou(bbox_pred[i], bbox_gt[i])
    avg_iou += iou
    if iou > iou_thresh:
        correct += 1
        
recall = correct/len(bbox_pred)
avg_iou = avg_iou/len(bbox_pred)
print('Recall:', recall, 'Avg_IOU:', avg_iou)

In [ ]:
print('%s UVD mean_l2_error: ' %data_split, mean_L2_error(uvd_gt, pred_uvd))
print('%s XYZ mean_l2_error: ' %data_split, mean_L2_error(xyz_gt, pred_xyz))
error = []
for i, (pred, uvd) in enumerate(zip(pred_uvd, uvd_gt)):
    error.append(mean_L2_error(uvd, pred))
error = np.asarray(error)
min_error_idx = np.argmin(error)
max_error_idx = np.argmax(error)
print('Best Pose id:', min_error_idx, 'uvd_error:', error[min_error_idx])
print('Worst Pose id:', max_error_idx, 'uvd_error:', error[max_error_idx])

In [ ]:
pck = percentage_frames_within_error_curve(xyz_gt, pred_xyz)
print(pck)
thresholds = np.arange(0, 85, 5)
print('AUC:', calc_auc(pck, thresholds))